In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pmdarima as pm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from prophet import Prophet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("train.csv", encoding="utf-8")

In [4]:
df['TimeSeries_Key'] = df['product_code'] + '_' + df['customer_code']

In [5]:
df['Week'] = df['calweek'].astype(str).str[-2:].astype(int)

In [6]:
df.columns

Index(['Cluster/Country/Region', 'SubCluster', 'calweek', 'brand', 'dc',
       'product_code', 'customer_code', 'Quantity', 'TimeSeries_Key', 'Week'],
      dtype='object')

In [7]:
df1 = df.drop(['Cluster/Country/Region', 'SubCluster', 'brand', 'dc'], axis = 1)

In [8]:
timeseries_key_values = list(df1['TimeSeries_Key'].unique())

In [9]:
# Convert 'calweek' column to datetime format
df1['calweek'] = pd.to_datetime(df1['calweek'].astype(str) + '1', format='%Y%W%w')

In [10]:
# Generate new columns for year, month, and quarter
df1["year"] = df1["calweek"].dt.year
df1["month"] = df1["calweek"].dt.month
df1["quarter"] = df1["calweek"].dt.quarter
# Generate a new column for season
df1['season'] = df1['calweek'].dt.month.apply(lambda x: (x%12 + 3)//3)

In [11]:
df1

,calweek,product_code,customer_code,Quantity,TimeSeries_Key,Week,year,month,quarter,season
0,2020-01-06,PRDCT1,CUSTA,537,PRDCT1_CUSTA,1,2020,1,1,1
1,2020-01-13,PRDCT1,CUSTA,544,PRDCT1_CUSTA,2,2020,1,1,1
2,2020-01-20,PRDCT1,CUSTA,574,PRDCT1_CUSTA,3,2020,1,1,1
3,2020-01-27,PRDCT1,CUSTA,523,PRDCT1_CUSTA,4,2020,1,1,1
4,2020-02-03,PRDCT1,CUSTA,571,PRDCT1_CUSTA,5,2020,2,1,1
...,...,...,...,...,...,...,...,...,...,...
18463,2023-03-13,PRDCT9,CUSTZ,1753,PRDCT9_CUSTZ,11,2023,3,1,2
18464,2023-03-20,PRDCT9,CUSTZ,1752,PRDCT9_CUSTZ,12,2023,3,1,2
18465,2023-03-27,PRDCT9,CUSTZ,1673,PRDCT9_CUSTZ,13,2023,3,1,2
18466,2023-04-03,PRDCT9,CUSTZ,1752,PRDCT9_CUSTZ,14,2023,4,2,2


In [12]:
start_week = 16
end_week = 28

filtered_df = df[(df['Week'] >= start_week) & (df['Week'] <= end_week)]

def generate_dataframes():
    grouped_df = filtered_df.groupby('TimeSeries_Key')
    for key, group in grouped_df:
        group['TimeSeries_Key'] = key
        yield group

combined_df = pd.concat(generate_dataframes(), ignore_index=True)
combined_df = combined_df.drop_duplicates(subset=['TimeSeries_Key', 'Week'])
combined_df = combined_df.drop(["product_code", "customer_code", "Quantity", "Week", "calweek"], axis = 1)
combined_df

,Cluster/Country/Region,SubCluster,brand,dc,TimeSeries_Key
0,US,SC1,BRANDA,DC1,PRDCT1_CUSTA
1,US,SC1,BRANDA,DC1,PRDCT1_CUSTA
2,US,SC1,BRANDA,DC1,PRDCT1_CUSTA
3,US,SC1,BRANDA,DC1,PRDCT1_CUSTA
4,US,SC1,BRANDA,DC1,PRDCT1_CUSTA
...,...,...,...,...,...
4181,US,SC2,BRANDC,DC4,PRDCT9_CUSTZ
4182,US,SC2,BRANDC,DC4,PRDCT9_CUSTZ
4183,US,SC2,BRANDC,DC4,PRDCT9_CUSTZ
4184,US,SC2,BRANDC,DC4,PRDCT9_CUSTZ


In [13]:
df_prod1 = df1[df1['TimeSeries_Key']=='PRDCT1_CUSTA']
df_prod1
# Group data by 'calweek' and aggregate 'Quantity'
ts_data = df_prod1.groupby('calweek')['Quantity'].sum()
ts_data
from statsmodels.tsa.stattools import adfuller
passing_data=adfuller(ts_data)
def adf_test(sales):
    result=adfuller(ts_data)
    labels = ['Test parameters', 'p-value','#Lags Used','Dataset observations']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("Dataset is stationary")
    else:
        print("Dataset is non-stationary ")
adf_test(ts_data)

Test parameters : -13.407568553264188
p-value : 4.439179540608158e-25
#Lags Used : 0
Dataset observations : 170
Dataset is stationary


In [14]:
# Create an empty dataframe to store MAPE values
results_df = pd.DataFrame()
results_mape = pd.DataFrame()

In [15]:
for i in timeseries_key_values:
    df_prod1 = df1[df1['TimeSeries_Key']==i]
    
    print("----------RESULT FOR",i,"----------")
    
    window_size = 3  # Define the window size for rolling average

    # Calculate rolling average for Quantity column
    df_prod1['Rolling_Average'] = df_prod1['Quantity'].rolling(window=window_size).mean()
    # Lagged Quantity
    df_prod1['lag1_Quantity'] = df_prod1['Quantity'].shift(1)
    
    # Split data into training and testing sets
    train_size = int(len(df1) * 0.8)  # Adjust the split ratio as per your requirement
    train_df = df1[:train_size]
    test_df = df1[train_size:]
    
    # Select the relevant columns for modeling
    X_train = train_df.drop(['calweek', 'product_code', 'customer_code', 'TimeSeries_Key', 'Quantity', 'season','Week'], axis=1)
    y_train = train_df['Quantity']
    X_test = test_df.drop(['calweek', 'product_code', 'customer_code', 'TimeSeries_Key', 'Quantity', 'season','Week'], axis=1)
    y_test = test_df['Quantity']
    
    # Find optimal p, d, q values using Auto ARIMA
    auto_arima_model = pm.auto_arima(y_train, seasonal=False, suppress_warnings=True)
    p, d, q = auto_arima_model.order
    
    # ARIMA model
    arima_model = ARIMA(endog=y_train, order=(p, d, q))  # ARIMA(p, d, q)
    arima_fit = arima_model.fit()
    arima_predictions = arima_fit.forecast(steps=len(X_test))

    # SARIMAX model
    sarimax_model = SARIMAX(endog=y_train, exog=X_train, order=(p, d, q))  # SARIMAX(p, d, q)
    sarimax_fit = sarimax_model.fit()
    sarimax_predictions = sarimax_fit.forecast(steps=len(X_test), exog=X_test)
    
    # Random Forest
    rf = RandomForestRegressor(n_estimators=100)
    rf.fit(X_train, y_train)
    rf_predictions = rf.predict(X_test)
    
    #ExtraTrees 
    et_extra_model = ExtraTreesRegressor(n_estimators=100)
    et_extra_model.fit(X_train, y_train)
    et_extra_predictions = et_extra_model.predict(X_test)
    
    # Gradient Boosting with XGBoost
    xgb_model = XGBRegressor(n_estimators=100)
    xgb_model.fit(X_train, y_train)
    xgb_predictions = xgb_model.predict(X_test)

    
    
    # Calculate MAPE
    def calculate_mape(actual, forecast):
        return np.mean(np.abs((actual - forecast) / actual)) * 100

    # Calculate MAPE for ARIMA
    arima_mape = calculate_mape(test_df['Quantity'], arima_predictions)

    # Calculate MAPE for SARIMA
    sarima_mape = calculate_mape(test_df['Quantity'], sarimax_predictions)
    
    # Calculate MAPE for RandomForest
    rf_mape = calculate_mape(test_df['Quantity'], rf_predictions)
    
    # Calculate MAPE for ExtraTrees
    et_extra_mape = calculate_mape(test_df['Quantity'], et_extra_predictions)
    
     # Calculate MAPE for Gradient Boosting with XGBoost
    xgb_mape = calculate_mape(test_df['Quantity'], xgb_predictions)    
    

    #Print MAPE values 
    print("ARIMA - MAPE: {:.2f}%".format(arima_mape))
    print("SARIMA - MAPE: {:.2f}%".format(sarima_mape))
    print("RANDOMFOREST - MAPE: {:.2f}%".format(rf_mape))
    print("ExtraTrees - MAPE: {:.2f}%".format(et_extra_mape))
    print("XGB - MAPE: {:.2f}%".format(xgb_mape))

     #FORECASTINGGG
    
    # Split data into training and testing sets
    train = df_prod1
    y_train = train['Quantity']
    
    # ARIMA model
    arima_model = ARIMA(endog=y_train, order=(1, 0, 0))  # ARIMA(p, d, q)
    arima_fit = arima_model.fit()
    arima_predictions = arima_fit.forecast(steps=13)
    
    #print(arima_predictions)
    # Convert arima_predictions to a list
    arima_predictions = arima_predictions.tolist()
    
    # Generate date values for the forecasted period
    forecast_dates = pd.date_range(start=df_prod1['calweek'].iloc[-1], periods=13, freq='W').strftime('%Y-%m-%d')

    for j in range(13):
        results_df = pd.concat([results_df, pd.DataFrame({
            'date': [forecast_dates[j]],
            'TimeSeries_Key': [i],
            'ARIMA_Predicted': [arima_predictions[j]]
             })], ignore_index=True)
        
        results_mape = pd.concat([results_mape, pd.DataFrame({
            'date': [forecast_dates[j]],
            'TimeSeries_Key': [i],
            'ARIMA_MAPE': [arima_mape],
            'SARIMA_MAPE': [sarima_mape],
            'RandomForest_MAPE': [rf_mape],
            'ExtraTrees_MAPE': [et_extra_mape],
            'XGB_MAPE': [xgb_mape],
        })], ignore_index=True) 
    


----------RESULT FOR PRDCT1_CUSTA ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.29%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT1_CUSTB ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.28%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT1_CUSTC ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.28%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT1_CUSTD ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.29%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT1_CUSTE ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.28%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT1_CUSTF ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.28%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT2_CUSTA 

ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.27%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT9_CUSTE ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.29%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT9_CUSTF ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.28%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT1_CUSTK ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.28%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT1_CUSTL ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.28%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT1_CUSTM ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.29%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT1_CUSTX ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE

ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.28%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT9_CUSTL ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.31%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT9_CUSTM ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.28%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT9_CUSTX ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.28%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT9_CUSTY ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.29%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%
----------RESULT FOR PRDCT9_CUSTZ ----------
ARIMA - MAPE: 19.43%
SARIMA - MAPE: 19.43%
RANDOMFOREST - MAPE: 25.28%
ExtraTrees - MAPE: 25.28%
XGB - MAPE: 25.28%


In [16]:
results_df

,date,TimeSeries_Key,ARIMA_Predicted
0,2023-04-16,PRDCT1_CUSTA,548.224299
1,2023-04-23,PRDCT1_CUSTA,548.084859
2,2023-04-30,PRDCT1_CUSTA,548.089462
3,2023-05-07,PRDCT1_CUSTA,548.089310
4,2023-05-14,PRDCT1_CUSTA,548.089315
...,...,...,...
1399,2023-06-11,PRDCT9_CUSTZ,1698.438596
1400,2023-06-18,PRDCT9_CUSTZ,1698.438596
1401,2023-06-25,PRDCT9_CUSTZ,1698.438596
1402,2023-07-02,PRDCT9_CUSTZ,1698.438596


In [17]:
results_mape

,date,TimeSeries_Key,ARIMA_MAPE,SARIMA_MAPE,RandomForest_MAPE,ExtraTrees_MAPE,XGB_MAPE
0,2023-04-16,PRDCT1_CUSTA,19.425963,19.430068,25.285162,25.282644,25.282644
1,2023-04-23,PRDCT1_CUSTA,19.425963,19.430068,25.285162,25.282644,25.282644
2,2023-04-30,PRDCT1_CUSTA,19.425963,19.430068,25.285162,25.282644,25.282644
3,2023-05-07,PRDCT1_CUSTA,19.425963,19.430068,25.285162,25.282644,25.282644
4,2023-05-14,PRDCT1_CUSTA,19.425963,19.430068,25.285162,25.282644,25.282644
...,...,...,...,...,...,...,...
1399,2023-06-11,PRDCT9_CUSTZ,19.425963,19.430068,25.283267,25.282644,25.282644
1400,2023-06-18,PRDCT9_CUSTZ,19.425963,19.430068,25.283267,25.282644,25.282644
1401,2023-06-25,PRDCT9_CUSTZ,19.425963,19.430068,25.283267,25.282644,25.282644
1402,2023-07-02,PRDCT9_CUSTZ,19.425963,19.430068,25.283267,25.282644,25.282644


In [18]:
# Convert the date column to datetime format
results_df['date'] = pd.to_datetime(results_df['date'])

# Convert the date format to 'YYYY-WW'
results_df['calweek'] = results_df['date'].dt.strftime('%Y-%U')

In [19]:
merged_df = pd.merge(results_df, combined_df, on='TimeSeries_Key')
merged_df.drop_duplicates(subset=['date', 'TimeSeries_Key'], inplace=True)
merged_df.reset_index(drop=True, inplace=True)
merged_df

,date,TimeSeries_Key,ARIMA_Predicted,calweek,Cluster/Country/Region,SubCluster,brand,dc
0,2023-04-16,PRDCT1_CUSTA,548.224299,2023-16,US,SC1,BRANDA,DC1
1,2023-04-23,PRDCT1_CUSTA,548.084859,2023-17,US,SC1,BRANDA,DC1
2,2023-04-30,PRDCT1_CUSTA,548.089462,2023-18,US,SC1,BRANDA,DC1
3,2023-05-07,PRDCT1_CUSTA,548.089310,2023-19,US,SC1,BRANDA,DC1
4,2023-05-14,PRDCT1_CUSTA,548.089315,2023-20,US,SC1,BRANDA,DC1
...,...,...,...,...,...,...,...,...
1399,2023-06-11,PRDCT9_CUSTZ,1698.438596,2023-24,US,SC2,BRANDC,DC4
1400,2023-06-18,PRDCT9_CUSTZ,1698.438596,2023-25,US,SC2,BRANDC,DC4
1401,2023-06-25,PRDCT9_CUSTZ,1698.438596,2023-26,US,SC2,BRANDC,DC4
1402,2023-07-02,PRDCT9_CUSTZ,1698.438596,2023-27,US,SC2,BRANDC,DC4
